In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from datasets import Dataset
from sklearn import metrics
import os

In [20]:
# Load datasets
train_df = pd.read_csv(
    'C:/Users/91898/Code/fibe/dataset_fibe/train.csv', encoding='ISO-8859-1')
test_df = pd.read_csv(
    'C:/Users/91898/Code/fibe/dataset_fibe/test.csv', encoding='ISO-8859-1')
sample_submission = pd.read_csv(
    'C:/Users/91898/Code/fibe/dataset_fibe/sample_submission.csv', encoding='ISO-8859-1')

KeyboardInterrupt: 

In [3]:
train_df.head()

,text,target,Word Count
0,"python courses python courses, python exercise...",academic interests,125
1,the learning point open digital education. a r...,academic interests,147
2,"tech news, latest technology, mobiles, laptops...",academic interests,143
3,the best it certification materials in usa | k...,academic interests,364
4,"bioland scientific, for your research needs bi...",academic interests,176


In [2]:
# Load the entire training dataset (no splitting)
from transformers import miniV2Tokenizer

from datasets import Dataset
from transformers import AutoTokenizer
train_df = pd.read_csv(
    'C:/Users/91898/Code/fibe/dataset_fibe/train.csv', encoding='ISO-8859-1')

# Preprocess the text data as done before
train_df['text'] = train_df['text'].str.lower().str.strip()

# Encode target labels using the same LabelEncoder
label_encoder = LabelEncoder()
train_df['target'] = label_encoder.fit_transform(train_df['target'])

# Tokenize the entire training dataset using the same tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')


def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)


# Convert to a Hugging Face dataset and apply tokenization
full_train_dataset = Dataset.from_pandas(train_df[['text', 'target']])
full_train_dataset = full_train_dataset.map(tokenize_function, batched=True)
full_train_dataset.set_format(
    type='torch', columns=['input_ids', 'attention_mask', 'target'])

Map:   0%|          | 0/697527 [00:00<?, ? examples/s]

In [4]:
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader

# Load your trained mini model
model = AutoModelForSequenceClassification.from_pretrained(
    'mini/saved_model', num_labels=len(label_encoder.classes_)
)
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a TrainingArguments object with FP16 enabled and a larger batch size
training_args = TrainingArguments(
    output_dir='.tettt/results',           # Output directory
    per_device_eval_batch_size=64,    # Increase this if your GPU has enough memory
    fp16=True                         # Enable mixed precision
)

# Initialize the Trainer with these arguments
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
)

# Make predictions on the full training set with gradient computation disabled
with torch.no_grad():
    mini_full_train_predictions = trainer.predict(full_train_dataset)

# Apply softmax to get probabilities
mini_full_train_probabilities = torch.nn.functional.softmax(
    torch.tensor(mini_full_train_predictions.predictions), dim=-1
).numpy()

# Save these probabilities
np.save('minilm_full_train_probabilities.npy',
        mini_full_train_probabilities)

c:\Users\91898\.conda\envs\tf\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/10899 [00:00<?, ?it/s]

In [4]:
# Preprocess the text data
def preprocess_text(text):
    return str(text).lower().strip()


# Apply preprocessing
train_df['text'] = train_df['text'].str.lower().str.strip()
test_df['text'] = test_df['text'].str.lower().str.strip()

# Encode target labels
label_encoder = LabelEncoder()
train_df['target'] = label_encoder.fit_transform(train_df['target'])

# Split into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['text'], train_df['target'], test_size=0.1, random_state=42
)

# Check the results of preprocessing and encoding
print(train_df.head())
print("Classes:", label_encoder.classes_)

                                                text  target  Word Count
0  python courses python courses, python exercise...       0         125
1  the learning point open digital education. a r...       0         147
2  tech news, latest technology, mobiles, laptops...       0         143
3  the best it certification materials in usa | k...       0         364
4  bioland scientific, for your research needs bi...       0         176
Classes: ['academic interests' 'arts and culture' 'automotives'
 'books and literature' 'business and finance' 'careers'
 'family and relationships' 'food and drinks' 'health' 'healthy living'
 'hobbies and interests' 'home and garden' 'movies' 'music and audio'
 'news and politics' 'personal finance' 'pets'
 'pharmaceuticals, conditions, and symptoms' 'real estate' 'shopping'
 'sports' 'style and fashion' 'technology and computing' 'television'
 'travel' 'video gaming']


In [5]:

tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')



# Convert pandas DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'labels': train_labels}))
val_dataset = Dataset.from_pandas(pd.DataFrame({'text': val_texts, 'labels': val_labels}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': test_df['text']}))

# Display first few entries to verify datasets
print(train_dataset[0])
print(val_dataset[0])
print(test_dataset[0])

{'text': 'lockdown perfect time tollywood stars introspect line up outside profession actor actress began drive back first love music shruti hasaan others like pranitha subhash turned help others tough times various opening samantha akkineni find out foretell horticulture growing bring about said last found something passionate break job starting get tired answering people asked hobby reply represent counterargument job hobby oh baby actress began journey first harvest cabbage microgreens lockdown carry insta explained one need declamatory lawn backyard gardening using space uncommitted domicile initially used window sill bedroom grow microgreens said interested growing require tray cocopeat seeds cool room used bedroom window lets sunlight partly also gave guide fans farsighted takes cum sprout number days tray needs covered use lamp case one enough sunlight room inspired tollywood diva take gardening first piazza pandemic fear able-bodied feed oneself cut back houses said often hear 

In [6]:
import os
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

train_dataset_path = 'mini/tokenized_train_dataset'
val_dataset_path = 'mini/tokenized_val_dataset'
test_dataset_path = 'mini/tokenized_test_dataset'

if os.path.exists(train_dataset_path) and os.path.exists(val_dataset_path) and os.path.exists(test_dataset_path):
    # Load the tokenized datasets
    train_dataset = load_from_disk(train_dataset_path)
    val_dataset = load_from_disk(val_dataset_path)
    test_dataset = load_from_disk(test_dataset_path)
    print("Tokenized datasets loaded successfully from disk.")
else:
    # Define the tokenization function
    def tokenize_function(examples):
        return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

    # Perform tokenization without multiprocessing
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Set the format for PyTorch
    train_dataset.set_format(type='torch', columns=[
                             'input_ids', 'attention_mask', 'labels'])
    val_dataset.set_format(type='torch', columns=[
                           'input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format(type='torch', columns=[
                            'input_ids', 'attention_mask'])

    # Save the tokenized datasets to disk
    os.makedirs('mini', exist_ok=True)
    train_dataset.save_to_disk(train_dataset_path)
    val_dataset.save_to_disk(val_dataset_path)
    test_dataset.save_to_disk(test_dataset_path)
    print("Tokenized datasets saved successfully to disk.")

# Check the results of tokenization
print(train_dataset[0])
print(val_dataset[0])

Tokenized datasets loaded successfully from disk.
{'labels': tensor(10), 'input_ids': tensor([  101,  5843,  7698,  3819,  2051,  9565, 26985,  3340, 17174, 13102,
        22471,  2240,  2039,  2648,  9518,  3364,  3883,  2211,  3298,  2067,
         2034,  2293,  2189, 14021, 22134,  2072,  2038, 14634,  2500,  2066,
        10975,  7088,  8322,  4942, 14949,  2232,  2357,  2393,  2500,  7823,
         2335,  2536,  3098, 11415, 17712,  4939, 18595,  2424,  2041, 18921,
        23567,  7570, 28228, 14561,  3652,  3288,  2055,  2056,  2197,  2179,
         2242, 13459,  3338,  3105,  3225,  2131,  5458, 10739,  2111,  2356,
        17792,  7514,  5050,  4675,  2906, 22850,  4765,  3105, 17792,  2821,
         3336,  3883,  2211,  4990,  2034, 11203, 28540, 12702, 28637,  3619,
         5843,  7698,  4287, 16021,  2696,  4541,  2028,  2342, 11703, 10278,
        14049, 10168, 16125, 21529,  2478,  2686,  4895,  9006, 22930,  3064,
        14383, 28775,  2571,  3322,  2109,  3332,  9033,

In [7]:
# Load BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('microsoft/MiniLM-L12-H384-uncased', num_labels=len(label_encoder.classes_))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set training arguments with mixed precision enabled
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='none',
    logging_steps=5000,
    evaluation_strategy="steps",
    save_steps=1000,
    eval_steps=1000,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision training
    report_to="none",
    resume_from_checkpoint=True 
)

# Display model details
print(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e

c:\Users\91898\.conda\envs\tf\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
# Define a function for computing metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = metrics.f1_score(labels, predictions, average='weighted')
    accuracy = metrics.accuracy_score(labels, predictions)
    return {'f1': f1, 'accuracy': accuracy}


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# Verify trainer configuration
print(trainer)

In [9]:
import torch
print(torch.cuda.is_available())  # Should output True

True


In [10]:
trainer.train(resume_from_checkpoint='./results/checkpoint-83000')

	logging_steps: 5000 (from args) != 500 (from trainer_state.json)


  0%|          | 0/117708 [00:00<?, ?it/s]

c:\Users\91898\.conda\envs\tf\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.4518, 'grad_norm': 5.853095054626465, 'learning_rate': 1.4609071053170434e-05, 'epoch': 2.13}
{'loss': 0.4329, 'grad_norm': 4.974795341491699, 'learning_rate': 1.4396201624462493e-05, 'epoch': 2.14}


  0%|          | 0/1090 [00:00<?, ?it/s]

...

  0%|          | 0/1090 [00:00<?, ?it/s]

{'eval_loss': 0.5094216465950012, 'eval_f1': 0.8610132360191208, 'eval_accuracy': 0.8609952260117845, 'eval_runtime': 182.8571, 'eval_samples_per_second': 381.462, 'eval_steps_per_second': 5.961, 'epoch': 2.22}
{'loss': 0.4149, 'grad_norm': 20.606487274169922, 'learning_rate': 1.2903556071257935e-05, 'epoch': 2.23}
{'loss': 0.44, 'grad_norm': 6.644088268280029, 'learning_rate': 1.2690260050508498e-05, 'epoch': 2.24}


  0%|          | 0/1090 [00:00<?, ?it/s]

{'eval_loss': 0.5002254843711853, 'eval_f1': 0.8615006775401592, 'eval_accuracy': 0.861167261623156, 'eval_runtime': 183.9291, 'eval_samples_per_second': 379.239, 'eval_steps_per_second': 5.926, 'epoch': 2.24}
{'train_runtime': 1739.4298, 'train_samples_per_second': 1082.724, 'train_steps_per_second': 67.67, 'train_loss': 0.024501816662875087, 'epoch': 2.24}


TrainOutput(global_step=88000, training_loss=0.024501816662875087, metrics={'train_runtime': 1739.4298, 'train_samples_per_second': 1082.724, 'train_steps_per_second': 67.67, 'total_flos': 9.3007825779499e+16, 'train_loss': 0.024501816662875087, 'epoch': 2.24283820980732})

In [11]:
# trainer.train()

In [12]:
# Make predictions on the test set
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

# Decode the predictions back to original labels
pred_labels = label_encoder.inverse_transform(pred_labels)

# Create the submission file
submission_df = pd.DataFrame({
    'Index': 'Article_' + test_df.index.astype(str),
    'target': pred_labels
})
submission_df.to_csv('mini2_submission.csv', index=False)

# Display the first few rows of the submission file
submission_df.head()

  0%|          | 0/2725 [00:00<?, ?it/s]

,Index,target
0,Article_0,technology and computing
1,Article_1,academic interests
2,Article_2,health
3,Article_3,academic interests
4,Article_4,academic interests


In [13]:
# Save the model and tokenizer
model.save_pretrained('mini/saved_model')
tokenizer.save_pretrained('mini/saved_tokenizer')
print("Model and tokenizer saved successfully.")

Model and tokenizer saved successfully.


In [14]:
# from transformers import AutoTokenizer
# import pandas as pd
# from tqdm import tqdm 

# # Load your data (adjust the path to your dataset)
# train_df = pd.read_csv(
#     'C:/Users/91898/Code/fibe/dataset_fibe/train.csv', encoding='ISO-8859-1')
# test_df = pd.read_csv(
#     'C:/Users/91898/Code/fibe/dataset_fibe/test.csv', encoding='ISO-8859-1')

# # Initialize the tokenizer (using the same model you are fine-tuning)
# tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

# # Combine text data from train and test for comprehensive analysis
# all_texts = pd.concat([train_df['text'], test_df['text']], ignore_index=True)

# # Add tqdm progress bar to the loop
# lengths = []
# for text in tqdm(all_texts, desc="Calculating average max_length", unit="text"):
#     length = len(tokenizer.encode(text, truncation=False))
#     lengths.append(length)

# # Calculate average max_length
# average_length = sum(lengths) / len(lengths)
# max_length = max(lengths)

# print(f"Average max_length: {average_length}")
# print(f"Max sequence length: {max_length}")

In [15]:
# Make predictions on the test set for MiniLM
minilm_predictions = trainer.predict(test_dataset)

# Save predicted probabilities
minilm_probabilities = torch.nn.functional.softmax(
    torch.tensor(minilm_predictions.predictions), dim=-1).numpy()
np.save('minilm_probabilities.npy', minilm_probabilities)

# Save the actual predicted labels
minilm_pred_labels = np.argmax(minilm_probabilities, axis=1)
np.save('minilm_pred_labels.npy', minilm_pred_labels)

  0%|          | 0/2725 [00:00<?, ?it/s]

In [17]:
# Make predictions on the training set for MiniLM
minilm_train_predictions = trainer.predict(train_dataset)

# Save predicted probabilities for the training set
minilm_train_probabilities = torch.nn.functional.softmax(
    torch.tensor(minilm_train_predictions.predictions), dim=-1).numpy()
np.save('minilm_train_probabilities.npy', minilm_train_probabilities)

# Save the actual predicted labels for the training set
minilm_train_pred_labels = np.argmax(minilm_train_probabilities, axis=1)
np.save('minilm_train_pred_labels.npy', minilm_train_pred_labels)

  0%|          | 0/9809 [00:00<?, ?it/s]

In [27]:
print(f"Length of train_dataset: {len(train_dataset)}")
print(f"Length of train_df: {len(train_df)}")

Length of train_dataset: 627774
Length of train_df: 627774


In [26]:
train_df = train_df.iloc[:len(train_dataset)]

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Load the saved probabilities from MiniLM
minilm_probabilities = np.load('minilm_train_probabilities.npy')

# Reload and align the train_df to match train_dataset
train_df = pd.read_csv(
    'C:/Users/91898/Code/fibe/dataset_fibe/train.csv', encoding='ISO-8859-1')
train_df = train_df.iloc[:len(minilm_probabilities)]

# Re-encode the actual target labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['target'])

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    minilm_probabilities, train_labels, test_size=0.1, random_state=42)

# Verify the shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (564996, 26), y_train shape: (564996,)
X_val shape: (62778, 26), y_val shape: (62778,)


In [46]:
# Import required libraries
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
import numpy as np


# Convert data into DMatrix with device specified
dtrain = xgb.DMatrix(X_train, label=y_train, nthread=-1)
dval = xgb.DMatrix(X_val, label=y_val, nthread=-1)

# Specify the parameters for the XGBoost model
params = {
    'objective': 'multi:softprob',  # multi-class classification
    'num_class': len(label_encoder.classes_),
    'tree_method': 'hist',
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'mlogloss',
    'verbosity': 1,
    'predictor': 'gpu_predictor',  # Use GPU for predictions
    'gpu_id': 0  # Assuming you want to use the first GPU
}

# Train the model with the DMatrix format
xgb_model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'eval')], early_stopping_rounds=20, verbose_eval=True)

# Make predictions on the validation set
y_val_pred = xgb_model.predict(dval)

# Calculate accuracy and F1 score
y_val_pred_labels = np.argmax(y_val_pred, axis=1)
accuracy = accuracy_score(y_val, y_val_pred_labels)
f1 = f1_score(y_val, y_val_pred_labels, average='weighted')

print(f"Blended Accuracy: {accuracy}")
print(f"Blended F1 Score: {f1}")

c:\Users\91898\.conda\envs\tf\lib\site-packages\xgboost\core.py:160: UserWarning: [14:10:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	eval-mlogloss:3.25298
[1]	eval-mlogloss:3.24860
[2]	eval-mlogloss:3.24478
[3]	eval-mlogloss:3.24156
[4]	eval-mlogloss:3.23876
[5]	eval-mlogloss:3.23636
[6]	eval-mlogloss:3.23427
[7]	eval-mlogloss:3.23251
[8]	eval-mlogloss:3.23091
[9]	eval-mlogloss:3.22949
[10]	eval-mlogloss:3.22827
[11]	eval-mlogloss:3.22722
[12]	eval-mlogloss:3.22632
[13]	eval-mlogloss:3.22567
[14]	eval-mlogloss:3.22498
[15]	eval-mlogloss:3.22439
[16]	eval-mlogloss:3.22392
[17]	eval-mlogloss:3.22355
[18]	eval-mlogloss:3.22319
[19]	eval-mlogloss:3.22289
[20]	eval-mlogloss:3.22267
[21]	eval-mlogloss:3.22243
[22]	eval-mlogloss:3.22233
[23]	eval-mlogloss:3.22217
[24]	eval-mlogloss:3.22211
[25]	eval-mlogloss:3.22201
[26]	eval-mlogloss:3.22193
[27]	eval-mlogloss:3.22184
[28]	eval-mlogloss:3.22190
[29]	eval-mlogloss:3.22188
[30]	eval-mlogloss:3.22192
[31]	eval-mlogloss:3.22187
[32]	eval-mlogloss:3.22188
[33]	eval-mlogloss:3.22201
[34]	eval-mlogloss:3.22212
[35]	eval-mlogloss:3.22219
[36]	eval-mlogloss:3.22238
[37]	eval-m

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb

# Convert data into DMatrix with device specified
dtrain = xgb.DMatrix(X_train, label=y_train, nthread=-1)
dval = xgb.DMatrix(X_val, label=y_val, nthread=-1)

# Specify the parameters for the XGBoost model
params = {
    'objective': 'multi:softprob',  # multi-class classification
    'num_class': len(label_encoder.classes_),
    'tree_method': 'hist',
    'max_depth': 6,  # Reduced max depth to prevent overfitting
    'learning_rate': 0.05,  # Lower learning rate
    'subsample': 0.7,  # Lower subsample to reduce overfitting
    'colsample_bytree': 0.7,  # Lower colsample_bytree
    'reg_alpha': 1,  # Add L1 regularization term
    'reg_lambda': 10,  # Add L2 regularization term
    'eval_metric': 'mlogloss',
    'verbosity': 1,
    'predictor': 'gpu_predictor',  # Use GPU for predictions
    'gpu_id': 0  # Assuming you want to use the first GPU
}

# Train the model with the DMatrix format, using early stopping
xgb_model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'eval')],
                      early_stopping_rounds=50,  # Stop if no improvement in 50 rounds
                      verbose_eval=10)

# Make predictions on the validation set
y_val_pred = xgb_model.predict(dval)

# Calculate accuracy and F1 score
y_val_pred_labels = np.argmax(y_val_pred, axis=1)
accuracy = accuracy_score(y_val, y_val_pred_labels)
f1 = f1_score(y_val, y_val_pred_labels, average='weighted')

print(f"Blended Accuracy: {accuracy}")
print(f"Blended F1 Score: {f1}")

NameError: name 'X_train' is not defined

In [50]:
# Predict on the validation set using MiniLM's output probabilities
from sklearn.metrics import accuracy_score, f1_score
minilm_val_pred = np.argmax(X_val, axis=1)

# Predict on the validation set using XGBoost
xgb_val_pred_prob = xgb_model.predict(xgb.DMatrix(X_val))
xgb_val_pred = np.argmax(xgb_val_pred_prob, axis=1)

# Perform blending (simple average)
final_blended_pred = np.argmax(
    (0.5 * X_val) + (0.5 * xgb_val_pred_prob), axis=1)

# Evaluate blended predictions
blended_accuracy = accuracy_score(y_val, final_blended_pred)
blended_f1 = f1_score(y_val, final_blended_pred, average='weighted')

print(f"Blended Accuracy: {blended_accuracy}")
print(f"Blended F1 Score: {blended_f1}")

Blended Accuracy: 0.043996304437860394
Blended F1 Score: 0.04406909023573695


In [51]:
# Load test dataset predictions from MiniLM
minilm_test_probabilities = np.load('minilm_probabilities.npy')

# Ensure minilm_test_probabilities is in the correct shape for XGBoost prediction
dtest = xgb.DMatrix(minilm_test_probabilities)

# Get XGBoost predictions
xgb_test_pred_prob = xgb_model.predict(dtest)

# Perform blending (simple average)
final_test_predictions = np.argmax(
    (0.5 * minilm_test_probabilities) + (0.5 * xgb_test_pred_prob), axis=1)

# Decode the final blended predictions back to original labels
final_test_labels = label_encoder.inverse_transform(final_test_predictions)

# Create a final submission file
submission_df = pd.DataFrame({
    'Index': 'Article_' + test_df.index.astype(str),
    'target': final_test_labels
})
submission_df.to_csv('blended_submission.csv', index=False)

print("Blending with XGBoost completed. Final predictions saved to 'blended_submission.csv'.")

Blending with XGBoost completed. Final predictions saved to 'blended_submission.csv'.
